# Exploração - Censo Escolar - Escola

**OBJETIVOS**: Explorar a base de Escola do censo escolar e fazer o rascunho do processamento de dados/

---

## Cabeçalho 

### Imports 

In [1]:
import os
import pyunpack
import rarfile
import shutil
import zipfile
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm import tqdm

### Caminhos 

In [4]:
PASTA_PROJETO = Path("C:/Users/pedro.forli/PycharmProjects/curso-ciencia-dados")
PASTA_DADOS = PASTA_PROJETO / "dados"
PASTA_SAIDA = PASTA_PROJETO / "saidas"
PASTA_NOTEBOOK = PASTA_PROJETO / "notebooks"
os.chdir(PASTA_PROJETO)

### Variáveis 

### Configurações 

In [5]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Código Próprio 

In [6]:
import src.aquisicao.inep.base_inep as etl_base_inep

---

## Carregamento de Dados 

In [147]:
dados = dict()

# para cada arquivo do censo demográfico
for censo in tqdm(os.listdir(PASTA_DADOS / "censo_escolar")):
    # abre o arquivo zip com o conteúdo do censo
    with zipfile.ZipFile(PASTA_DADOS / f"censo_escolar/{censo}") as z:
        # lista os conteúdos dos arquivos zip que contém o nome "escolas"
        arqs = [f for f in z.namelist() if "escolas." in f.lower()]
        
        # se houver algum arquivo deste tipo dentro do zip
        if len(arqs) == 1:
            arq = arqs[0]
            
            # e este arquivo for um CSV
            if ".csv" in arq.lower():
                # le os conteúdos do arquivo por meio do buffer do zip
                dados[censo] = pd.read_csv(z.open(arq), encoding="latin-1", sep="|")
            
            # caso seja outro arquivo zip
            elif ".zip" in arq.lower():
                # cria um novo zipfile e le o arquivo deste novo zip
                with zipfile.ZipFile(z.open(arq)) as z2:
                    arq = z2.namelist()[0]
                    dados[censo] = pd.read_csv(z2.open(arq), encoding="latin-1", sep="|")
            
            # caso seja um arquivo winrar
            elif ".rar" in arq.lower():
                # extraí o conteúdo do arquivo
                z.extract(arq, path=PASTA_DADOS / "censo_escolar")
                (
                    pyunpack.Archive(PASTA_DADOS / f"censo_escolar/{arq}")
                    .extractall(PASTA_DADOS / "censo_escolar")
                )
                
                # lê os dados do disco
                csv = [f for f in os.listdir(PASTA_DADOS / "censo_escolar") if "escolas." in f.lower()][0]
                dados[censo] = pd.read_csv(PASTA_DADOS / f"censo_escolar/{csv}", encoding="latin-1", sep="|")
                
                # excluí os conteúdos extraídos
                shutil.rmtree(PASTA_DADOS / f"censo_escolar/{arq.split('/')[0]}")
                os.remove(PASTA_DADOS / f"censo_escolar/{csv}")

  0%|                                                                                 | 0/26 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\envs\curso-ciencia-dados\lib\site-packages\IPython\core\interactiveshell.py:3417: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
 77%|███████████████████████████████████████████████████████▍                | 20/26 [00:15<00:09,  1.52s/it]C:\ProgramData\Anaconda3\envs\curso-ciencia-dados\lib\site-packages\IPython\core\interactiveshell.py:3417: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
 81%|██████████████████████████████████████████████████████████▏             | 21/26 [00:17<00:08,  1.75s/it]C:\ProgramData\Anaconda3\envs\curso-ciencia-dados\lib\site-packages\IPython\core\interactiveshell.py:3417: DtypeWarning: Columns (3,5,6) have mixed types.S

---

## Exploração 

Visualiza como os dados aparecem na base

In [6]:
dados["2020.zip"].head()

,NU_ANO_CENSO,CO_ENTIDADE,NO_ENTIDADE,CO_ORGAO_REGIONAL,TP_SITUACAO_FUNCIONAMENTO,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,...,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF
0,2020,11000023,EEEE ABNAEL MACHADO DE LIMA - CENE,00009,1,06/02/2020,15/12/2020,1,1101,11001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020,11000040,EMEIEF PEQUENOS TALENTOS,00009,1,06/02/2020,23/12/2020,1,1101,11001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020,11000058,CENTRO DE ENSINO CLASSE A,00009,1,03/02/2020,11/12/2020,1,1101,11001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020,11000082,CENTRO EDUCACIONAL PRESBITERIANO 15 DE NOVEMBRO,00009,1,03/02/2020,04/12/2020,1,1101,11001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020,11000104,CENTRO EDUC CORA CORALINA,00009,1,03/02/2020,15/12/2020,1,1101,11001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
dados["2020.zip"].tail()

,NU_ANO_CENSO,CO_ENTIDADE,NO_ENTIDADE,CO_ORGAO_REGIONAL,TP_SITUACAO_FUNCIONAMENTO,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,...,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF
224224,2020,53084071,ESC DIVINO MESTRE,99.0,2,NaN,NaN,5,5301,53001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224225,2020,53085000,COBIAN - COL BIANGULO,99.0,1,27/01/2020,23/12/2020,5,5301,53001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
224226,2020,53086007,CED VITORIA REGIA,99.0,1,27/01/2020,18/12/2020,5,5301,53001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
224227,2020,53087003,CED SANTOS DUMONT - CESAN,99.0,2,NaN,NaN,5,5301,53001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224228,2020,53088034,OBRA SOCIAL NOSSA SENHORA DE FATIMA,99.0,2,NaN,NaN,5,5301,53001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
dados["2020.zip"].sample(5)

,NU_ANO_CENSO,CO_ENTIDADE,NO_ENTIDADE,CO_ORGAO_REGIONAL,TP_SITUACAO_FUNCIONAMENTO,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,...,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF
144207,2020,33124884,CENTRO EDUC ESTRELA GUIA LTDA,11006,2,NaN,NaN,3,3306,33018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130425,2020,31353450,EE FELICIANO MIGUEL ABDALLA,4.0,1,10/02/2020,13/12/2020,3,3108,31040,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74805,2020,26111349,ESCOLA SUPERVISORA MIRIAM SEIXAS,4.0,1,03/02/2020,22/12/2020,2,2605,26017,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
111888,2020,31056332,EM - PROFESSORA JOSEFA AZEVEDO TORRES,37.0,1,05/02/2020,16/12/2020,3,3110,31052,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60510,2020,24047643,ESC MUNICIPAL MANOEL ADELINO,00003,1,09/03/2020,18/12/2020,2,2403,24015,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Verifica informações básicas de tamanho

In [9]:
dados["2020.zip"].shape

(224229, 238)

Tipagem

In [10]:
dados["2020.zip"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224229 entries, 0 to 224228
Columns: 238 entries, NU_ANO_CENSO to IN_ESP_EXCLUSIVA_PROF
dtypes: float64(223), int64(11), object(4)
memory usage: 407.2+ MB


E conteído

In [11]:
dados["2020.zip"].columns

Index(['NU_ANO_CENSO', 'CO_ENTIDADE', 'NO_ENTIDADE', 'CO_ORGAO_REGIONAL',
       'TP_SITUACAO_FUNCIONAMENTO', 'DT_ANO_LETIVO_INICIO',
       'DT_ANO_LETIVO_TERMINO', 'CO_REGIAO', 'CO_MESORREGIAO',
       'CO_MICRORREGIAO',
       ...
       'IN_ESP_EXCLUSIVA_MEDIO_INTEGR', 'IN_ESP_EXCLUSIVA_MEDIO_NORMAL',
       'IN_COMUM_EJA_FUND', 'IN_COMUM_EJA_MEDIO', 'IN_COMUM_EJA_PROF',
       'IN_ESP_EXCLUSIVA_EJA_FUND', 'IN_ESP_EXCLUSIVA_EJA_MEDIO',
       'IN_ESP_EXCLUSIVA_EJA_PROF', 'IN_COMUM_PROF', 'IN_ESP_EXCLUSIVA_PROF'],
      dtype='object', length=238)

Tipos de variáveis

- IN = Indicador = flag
- NU = Número
- NO = Nome
- DT = Data
- CO = Código
- TP = Tipo
- QT = Quantidade

Faz uma contagem de linhas

In [13]:
dados["2020.zip"].count()

NU_ANO_CENSO                  224229
CO_ENTIDADE                   224229
NO_ENTIDADE                   224229
CO_ORGAO_REGIONAL             218075
TP_SITUACAO_FUNCIONAMENTO     224229
                               ...  
IN_ESP_EXCLUSIVA_EJA_FUND     181279
IN_ESP_EXCLUSIVA_EJA_MEDIO    181279
IN_ESP_EXCLUSIVA_EJA_PROF     181279
IN_COMUM_PROF                 181279
IN_ESP_EXCLUSIVA_PROF         181279
Length: 238, dtype: int64

Aparentemente há algumas variáveis com mais o menos conteúdo dependendo das condições de construção daquela variável

In [14]:
[
    c for c in dados["2020.zip"].columns 
    if dados["2020.zip"][c].count() != dados["2020.zip"].shape[0] 
    and dados["2020.zip"][c].count() != dados["2020.zip"]["TP_SITUACAO_FUNCIONAMENTO"].value_counts().iloc[0]
]

['CO_ORGAO_REGIONAL',
 'IN_VINCULO_SECRETARIA_EDUCACAO',
 'IN_VINCULO_SEGURANCA_PUBLICA',
 'IN_VINCULO_SECRETARIA_SAUDE',
 'IN_VINCULO_OUTRO_ORGAO',
 'TP_CATEGORIA_ESCOLA_PRIVADA',
 'IN_CONVENIADA_PP',
 'TP_CONVENIO_PODER_PUBLICO',
 'IN_MANT_ESCOLA_PRIVADA_EMP',
 'IN_MANT_ESCOLA_PRIVADA_ONG',
 'IN_MANT_ESCOLA_PRIVADA_OSCIP',
 'IN_MANT_ESCOLA_PRIV_ONG_OSCIP',
 'IN_MANT_ESCOLA_PRIVADA_SIND',
 'IN_MANT_ESCOLA_PRIVADA_SIST_S',
 'IN_MANT_ESCOLA_PRIVADA_S_FINS',
 'TP_RESPONSAVEL_REGULAMENTACAO',
 'CO_ESCOLA_SEDE_VINCULADA',
 'CO_IES_OFERTANTE',
 'TP_OCUPACAO_PREDIO_ESCOLAR',
 'TP_OCUPACAO_GALPAO',
 'IN_PREDIO_COMPARTILHADO',
 'IN_BANDA_LARGA',
 'IN_SERIE_ANO',
 'IN_PERIODOS_SEMESTRAIS',
 'IN_FUNDAMENTAL_CICLOS',
 'IN_GRUPOS_NAO_SERIADOS',
 'IN_MODULOS',
 'IN_FORMACAO_ALTERNANCIA',
 'TP_INDIGENA_LINGUA',
 'CO_LINGUA_INDIGENA_1',
 'CO_LINGUA_INDIGENA_2',
 'CO_LINGUA_INDIGENA_3',
 'IN_RESERVA_PPI',
 'IN_RESERVA_RENDA',
 'IN_RESERVA_PUBLICA',
 'IN_RESERVA_PCD',
 'IN_RESERVA_OUTROS',
 'IN_RESERVA

---

## Tratamento de Dados 

### Renames 

In [148]:
for base in dados.values():
    base.rename(
        columns={
            "ANO_CENSO": "NU_ANO_CENSO",
            "PK_COD_ENTIDADE": "CO_ENTIDADE",
            "COD_ORGAO_REGIONAL_INEP": "CO_ORGAO_REGIONAL",
            "DESC_SITUACAO_FUNCIONAMENTO": "TP_SITUACAO_FUNCIONAMENTO",
            "SIGLA": "SIGLA_UF",
            "FK_COD_ESTADO": "CO_UF",
            "FK_COD_MUNICIPIO": "CO_MUNICIPIO",
            "FK_COD_DISTRITO": "CO_DISTRITO",
            "ID_DEPENDENCIA_ADM": "TP_DEPENDENCIA",
            "ID_LOCALIZACAO": "TP_LOCALIZACAO",
            "DESC_CATEGORIA_ESCOLA_PRIVADA": "TP_CATEGORIA_ESCOLA_PRIVADA",
            "ID_CONVENIADA_PP": "IN_CONVENIADA_PP",
            "ID_TIPO_CONVENIO_PODER_PUBLICO": "TP_CONVENIO_PODER_PUBLICO",
            "ID_MANT_ESCOLA_PRIVADA_EMP": "IN_MANT_ESCOLA_PRIVADA_EMP",
            "ID_MANT_ESCOLA_PRIVADA_ONG": "IN_MANT_ESCOLA_PRIVADA_ONG",
            "ID_MANT_ESCOLA_PRIVADA_SIND": "IN_MANT_ESCOLA_PRIVADA_SIND",
            "ID_MANT_ESCOLA_PRIVADA_SIST_S": "IN_MANT_ESCOLA_PRIVADA_SIST_S",
            "ID_MANT_ESCOLA_PRIVADA_S_FINS": "IN_MANT_ESCOLA_PRIVADA_S_FINS",
            "ID_DOCUMENTO_REGULAMENTACAO": "TP_REGULAMENTACAO",
            "ID_LOCAL_FUNC_PREDIO_ESCOLAR": "TP_OCUPACAO_PREDIO_ESCOLAR",
            "ID_LOCAL_FUNC_SALAS_EMPRESA": "IN_LOCAL_FUNC_SALAS_EMPRESA",
            "ID_LOCAL_FUNC_SOCIOEDUCATIVA": "IN_LOCAL_FUNC_SOCIOEDUCATIVO",
            "ID_LOCAL_FUNC_UNID_PRISIONAL": "IN_LOCAL_FUNC_UNID_PRISIONAL",
            "ID_LOCAL_FUNC_TEMPLO_IGREJA": "IN_LOCAL_FUNC_TEMPLO_IGREJA",
            "ID_LOCAL_FUNC_CASA_PROFESSOR": "IN_LOCAL_FUNC_CASA_PROFESSOR",
            "ID_LOCAL_FUNC_GALPAO": "TP_OCUPACAO_GALPAO",
            "ID_LOCAL_FUNC_OUTROS": "IN_LOCAL_FUNC_OUTROS",
            "ID_LOCAL_FUNC_SALAS_OUTRA_ESC": "IN_LOCAL_FUNC_SALAS_OUTRA_ESC",
            "ID_ESCOLA_COMP_PREDIO": "IN_PREDIO_COMPARTILHADO",
            "ID_AGUA_FILTRADA": "IN_AGUA_FILTRADA",
            "ID_AGUA_REDE_PUBLICA": "IN_AGUA_REDE_PUBLICA",
            "ID_AGUA_POCO_ARTESIANO": "IN_AGUA_POCO_ARTESIANO",
            "ID_AGUA_CACIMBA": "IN_AGUA_CACIMBA",
            "ID_AGUA_FONTE_RIO": "IN_AGUA_FONTE_RIO",
            "ID_AGUA_INEXISTENTE": "IN_AGUA_INEXISTENTE",
            "ID_ENERGIA_REDE_PUBLICA": "IN_ENERGIA_REDE_PUBLICA",
            "ID_ENERGIA_GERADOR": "IN_ENERGIA_GERADOR",
            "ID_ENERGIA_OUTROS": "IN_ENERGIA_OUTROS",
            "ID_ENERGIA_INEXISTENTE": "IN_ENERGIA_INEXISTENTE",
            "ID_ESGOTO_REDE_PUBLICA": "IN_ESGOTO_REDE_PUBLICA",
            "ID_ESGOTO_FOSSA": "IN_ESGOTO_FOSSA",
            "ID_ESGOTO_INEXISTENTE": "IN_ESGOTO_INEXISTENTE",
            "ID_LIXO_COLETA_PERIODICA": "IN_LIXO_SERVICO_COLETA",
            "ID_LIXO_QUEIMA": "IN_LIXO_QUEIMA",
            "ID_LIXO_JOGA_OUTRA_AREA": "IN_LIXO_DESCARTA_OUTRA_AREA",
            "ID_LIXO_RECICLA": "IN_TRATAMENTO_LIXO_RECICLAGEM",
            "ID_LIXO_ENTERRA": "IN_LIXO_ENTERRA",
            "ID_LIXO_OUTROS": "IN_LIXO_DESTINO_FINAL_PUBLICO",
            "IN_LIXO_OUTROS": "IN_LIXO_DESTINO_FINAL_PUBLICO",
            "ID_SALA_DIRETORIA": "IN_SALA_DIRETORIA",
            "ID_SALA_PROFESSOR": "IN_SALA_PROFESSOR",
            "ID_LABORATORIO_INFORMATICA": "IN_LABORATORIO_INFORMATICA",
            "ID_LABORATORIO_CIENCIAS": "IN_LABORATORIO_CIENCIAS",
            "ID_SALA_ATENDIMENTO_ESPECIAL": "IN_SALA_ATENDIMENTO_ESPECIAL",
            "ID_QUADRA_ESPORTES_COBERTA": "IN_QUADRA_ESPORTES_COBERTA",
            "ID_QUADRA_ESPORTES_DESCOBERTA": "IN_QUADRA_ESPORTES_DESCOBERTA",
            "ID_COZINHA": "IN_COZINHA",
            "ID_BIBLIOTECA": "IN_BIBLIOTECA",
            "ID_SALA_LEITURA": "IN_SALA_LEITURA",
            "ID_PARQUE_INFANTIL": "IN_PARQUE_INFANTIL",
            "ID_BERCARIO": "IN_BERCARIO",
            "ID_SANITARIO_FORA_PREDIO": "IN_BANHEIRO_FORA_PREDIO",
            "ID_SANITARIO_DENTRO_PREDIO": "IN_BANHEIRO_DENTRO_PREDIO",
            "ID_SANITARIO_EI": "IN_BANHEIRO_EI",
            "ID_SANITARIO_PNE": "IN_BANHEIRO_PNE",
            "ID_DEPENDENCIAS_PNE": "IN_DEPENDENCIAS_PNE",
            "ID_SECRETARIA": "IN_SECRETARIA",
            "ID_BANHEIRO_CHUVEIRO": "IN_BANHEIRO_CHUVEIRO",
            "ID_REFEITORIO": "IN_REFEITORIO",
            "ID_DESPENSA": "IN_DESPENSA",
            "ID_ALMOXARIFADO": "IN_ALMOXARIFADO",
            "ID_AUDITORIO": "IN_AUDITORIO",
            "ID_PATIO_COBERTO": "IN_PATIO_COBERTO",
            "ID_PATIO_DESCOBERTO": "IN_PATIO_DESCOBERTO",
            "ID_ALOJAM_ALUNO": "IN_DORMITORIO_ALUNO",
            "ID_ALOJAM_PROFESSOR": "IN_DORMITORIO_PROFESSOR",
            "ID_AREA_VERDE": "IN_AREA_VERDE",
            "ID_LAVANDERIA": "IN_LAVANDERIA",
            "ID_DEPENDENCIAS_OUTRAS": "IN_DEPENDENCIAS_OUTRAS",
            "NUM_SALAS_EXISTENTES": "QT_SALAS_EXISTENTES",
            "NUM_SALAS_UTILIZADAS": "QT_SALAS_UTILIZADAS",
            "NUM_EQUIP_TV": "QT_EQUIP_TV",
            "NUM_EQUIP_VIDEOCASSETE": "QT_EQUIP_VIDEOCASSETE",
            "NUM_EQUIP_DVD": "QT_EQUIP_DVD",
            "NUM_EQUIP_PARABOLICA": "QT_EQUIP_PARABOLICA",
            "NUM_EQUIP_COPIADORA": "QT_EQUIP_COPIADORA",
            "NUM_EQUIP_RETRO": "QT_EQUIP_RETRO",
            "NUM_EQUIP_IMPRESSORA": "QT_EQUIP_IMPRESSORA",
            "NUM_EQUIP_SOM": "QT_EQUIP_SOM",
            "NUM_EQUIP_MULTIMIDIA": "QT_EQUIP_MULTIMIDIA",
            "NUM_EQUIP_FAX": "QT_EQUIP_FAX",
            "NUM_EQUIP_FOTO": "QT_EQUIP_FOTO",
            "NUM_COMPUTADORES": "QT_DESKTOP",
            "NUM_COMP_ADMINISTRATIVOS": "QT_DESKTOP_ADM",
            "NUM_COMP_ALUNOS": "QT_DESKTOP_ALUNO",
            "ID_INTERNET": "IN_INTERNET",
            "ID_BANDA_LARGA": "IN_BANDA_LARGA",
            "NUM_FUNCIONARIOS": "QT_FUNCIONARIOS",
            "ID_ALIMENTACAO": "IN_ALIMENTACAO",
            "ID_AEE": "TP_AEE",
            "ID_MOD_ATIV_COMPLEMENTAR": "TP_ATIVIDADE_COMPLEMENTAR",
            "ID_MOD_ENS_REGULAR": "IN_REGULAR",
            "ID_REG_INFANTIL_CRECHE": "IN_COMUM_CRECHE",
            "ID_REG_INFANTIL_PREESCOLA": "IN_COMUM_PRE",
            "ID_REG_FUND_8_ANOS": "IN_COMUM_FUND_AI",
            "ID_REG_FUND_9_ANOS": "IN_COMUM_FUND_AF",
            "ID_REG_MEDIO_MEDIO": "IN_COMUM_MEDIO_MEDIO",
            "ID_REG_MEDIO_INTEGRADO": "IN_COMUM_MEDIO_INTEGRADO",
            "ID_REG_MEDIO_NORMAL": "IN_COMUM_MEDIO_NORMAL",
            "ID_ESP_INFANTIL_CRECHE": "IN_ESP_EXCLUSIVA_CRECHE",
            "ID_ESP_INFANTIL_PREESCOLA": "IN_ESP_EXCLUSIVA_PRE",
            "ID_ESP_FUND_8_ANOS": "IN_ESP_EXCLUSIVA_FUND_AI",
            "ID_ESP_FUND_9_ANOS": "IN_ESP_EXCLUSIVA_FUND_AF",
            "ID_ESP_MEDIO_MEDIO": "IN_ESP_EXCLUSIVA_MEDIO_MEDIO",
            "ID_ESP_MEDIO_INTEGRADO": "IN_ESP_EXCLUSIVA_MEDIO_INTEGR",
            "ID_ESP_MEDIO_NORMAL": "IN_ESP_EXCLUSIVA_MEDIO_NORMAL",
            "ID_ESP_EJA_FUNDAMENTAL": "IN_ESP_EXCLUSIVA_EJA_FUND",
            "ID_ESP_EJA_MEDIO": "IN_ESP_EXCLUSIVA_EJA_MEDIO",
            "ID_MOD_EJA": "IN_EJA",
            "ID_EJA_FUNDAMENTAL": "IN_EJA_FUNDAMENTAL",
            "ID_EJA_MEDIO": "IN_EJA_MEDIO",
            "ID_EJA_PROJOVEM": "IN_EJA_PROJOVEM",
            "ID_FUND_CICLOS": "IN_FUNDAMENTAL_CICLOS",
            "ID_LOCALIZACAO_DIFERENCIADA": "TP_LOCALIZACAO_DIFERENCIADA",
            "ID_MATERIAL_ESP_NAO_UTILIZA": "IN_MATERIAL_PED_NENHUM",
            "ID_MATERIAL_ESP_QUILOMBOLA": "IN_MATERIAL_PED_ETNICO",
            "ID_MATERIAL_ESP_INDIGENA": "IN_MATERIAL_PED_INDIGENA",
            "ID_MATERIAL_ESP_CEGOS": "IN_MATERIAL_PED_CEGOS",
            "ID_MATERIAL_ESP_ETNICO": "IN_MATERIAL_PED_ETNICO",
            "ID_MATERIAL_ESP_INDIGINA": "IN_MATERIAL_PED_INDIGENA",
            "ID_MATERIAL_ESP_QUILOMBOLAS": "IN_MATERIAL_PED_ETNICO",
            "ID_MATERIAL_ESP_SURDOS": "IN_MATERIAL_PED_SURDOS",
            "ID_EDUCACAO_INDIGENA": "IN_EDUCACAO_INDIGENA",
            "ID_LINGUA_INDIGENA": "IN_LINGUA_INDIGENA",
            "FK_COD_LINGUA_INDIGENA": "CO_LINGUA_INDIGENA_1",
            "ID_LINGUA_PORTUGUESA": "IN_LINGUA_PORTUGUESA",
            "ID_ESPACO_TURMA_PBA": "IN_BRASIL_ALFABETIZADO",
            "ID_ABRE_FINAL_SEMANA": "IN_FINAL_SEMANA",
            "ID_PROPOSTA_PEDAG_ALTERNANCIA": "IN_FORMACAO_ALTERNANCIA",
            "COD_ORGAO_REGIONAL": "CO_ORGAO_REGIONAL",
            "CO_LINGUA_INDIGENA": "CO_LINGUA_INDIGENA_1",
            "ID_EQUIP_COPIADORA": "IN_EQUIP_COPIADORA",
            "ID_EQUIP_DVD": "IN_EQUIP_DVD",
            "ID_EQUIP_FAX": "IN_EQUIP_FAX",
            "ID_EQUIP_FOTO": "IN_EQUIP_FOTO",
            "ID_EQUIP_IMPRESSORA": "IN_EQUIP_IMPRESSORA",
            "ID_EQUIP_IMPRESSORA_BRAILE": "IN_EQUIP_IMPRESSORA_BRAILE",
            "ID_EQUIP_MULTIMIDIA": "IN_EQUIP_MULTIMIDIA",
            "ID_EQUIP_PARABOLICA": "IN_EQUIP_PARABOLICA",
            "ID_EQUIP_SOM": "IN_EQUIP_SOM",
            "ID_EQUIP_TV": "IN_EQUIP_TV",
            "ID_EQUIP_VIDEOCASSETE": "IN_EQUIP_VIDEOCASSETE",
            "ID_COMPUTADORES": "IN_DESKTOP",
            "IN_COMPUTADOR": "IN_DESKTOP",
            "ID_ESP_PROFISSIONAL": "IN_PROFISSIONALIZANTE",
            "NU_COMPUTADOR": "QT_DESKTOP",
            "NU_COMP_ADMINISTRATIVO": "QT_DESKTOP_ADM",
            "NU_COMP_ALUNO": "QT_DESKTOP_ALUNO",
            "NU_EQUIP_COPIADORA": "QT_EQUIP_COPIADORA",
            "NU_EQUIP_DVD": "QT_EQUIP_DVD",
            "NU_EQUIP_FAX": "QT_EQUIP_FAX",
            "NU_EQUIP_FOTO": "QT_EQUIP_FOTO",
            "NU_EQUIP_IMPRESSORA": "QT_EQUIP_IMPRESSORA",
            "NU_EQUIP_IMPRESSORA_MULT": "QT_EQUIP_IMPRESSORA_MULT",
            "NU_EQUIP_MULTIMIDIA": "QT_EQUIP_MULTIMIDIA",
            "NU_EQUIP_PARABOLICA": "QT_EQUIP_PARABOLICA",
            "NU_EQUIP_RETROPROJETOR": "QT_EQUIP_RETRO",
            "NU_EQUIP_SOM": "QT_EQUIP_SOM",
            "NU_EQUIP_TV": "QT_EQUIP_TV",
            "NU_EQUIP_VIDEOCASSETE": "QT_EQUIP_VIDEOCASSETE",
            "NU_FUNCIONARIOS": "QT_FUNCIONARIOS",
            "QT_COMPUTADOR": "QT_DESKTOP",
            "QT_COMP_ADMINISTRATIVO": "QT_DESKTOP_ADM",
            "QT_COMP_ALUNO": "QT_DESKTOP_ALUNO",
            "QT_EQUIP_RETROPROJETOR": "QT_EQUIP_RETRO",
            "IN_ALOJAM_ALUNO": "IN_DORMITORIO_ALUNO",
            "IN_ALOJAM_PROFESSOR": "IN_DORMITORIO_PROFESSOR",
            "ID_EQUIP_RETRO": "IN_EQUIP_RETRO",
            "IN_EQUIP_RETROPROJETOR": "IN_EQUIP_RETRO",
            "IN_LIXO_COLETA_PERIODICA": "IN_LIXO_SERVICO_COLETA",
            "IN_LIXO_JOGA_OUTRA_AREA": "IN_LIXO_DESCARTA_OUTRA_AREA",
            "IN_LIXO_RECICLA": "IN_TRATAMENTO_LIXO_RECICLAGEM",            
            "IN_MATERIAL_ESP_INDIGENA": "IN_MATERIAL_PED_INDIGENA",
            "IN_MATERIAL_ESP_NAO_UTILIZA": "IN_MATERIAL_PED_NENHUM",
            "IN_MATERIAL_ESP_QUILOMBOLA": "IN_MATERIAL_PED_ETNICO",
            "IN_MATERIAL_PED_CEGOS": "IN_MATERIAL_PED_CEGOS",
            "IN_MATERIAL_PED_SURDOS": "IN_MATERIAL_PED_SURDOS",
            "NU_SALAS_EXISTENTES": "QT_SALAS_EXISTENTES",
            "NU_SALAS_UTILIZADAS": "QT_SALAS_UTILIZADAS",
            "ID_EJA_ORGANIZACAO_ANUAL": "IN_EJA_ANUAL",
            "ID_EJA_ORGANIZACAO_SEM": "IN_EJA_SEM",
            "ID_ESP_MEDIO_PROFISSIONAL": "IN_ESP_EXCLUSIVA_PROF",
            "ID_LOCAL_FUNC_PRISIONAL": "IN_LOCAL_FUNC_UNID_PRISIONAL",
            "ID_MANT_ESCOLA_PRIVADA_APAE": "IN_MANT_ESCOLA_PRIVADA_S_FINS",
            "ID_MOD_ENS_ESP": "IN_ESPECIAL_EXCLUSIVA",
            "ID_QUADRA_ESPORTES": "IN_QUADRA_ESPORTES",
            "ID_REG_MEDIO_PROF": "IN_COMUM_PROF",
        },
        inplace=True
    )

### Drop 

In [149]:
# coluans que poderão ser extraídas da base de turmas
cols_turma = [
    "TP_AEE",
    "TP_ATIVIDADE_COMPLEMENTAR",
    "IN_MEDIACAO_PRESENCIAL",
    "IN_MEDIACAO_SEMIPRESENCIAL",
    "IN_MEDIACAO_EAD",
    "IN_ESPECIAL_EXCLUSIVA",
    "IN_REGULAR",
    "IN_EJA",
    "IN_PROFISSIONALIZANTE",
    "IN_COMUM_CRECHE",
    "IN_COMUM_PRE",
    "IN_COMUM_FUND_AI",
    "IN_COMUM_FUND_AF",
    "IN_COMUM_MEDIO_MEDIO",
    "IN_COMUM_MEDIO_INTEGRADO",
    "IN_COMUM_MEDIO_NORMAL",
    "IN_ESP_EXCLUSIVA_CRECHE",
    "IN_ESP_EXCLUSIVA_PRE",
    "IN_ESP_EXCLUSIVA_FUND_AI",
    "IN_ESP_EXCLUSIVA_FUND_AF",
    "IN_ESP_EXCLUSIVA_MEDIO_MEDIO",
    "IN_ESP_EXCLUSIVA_MEDIO_INTEGR",
    "IN_ESP_EXCLUSIVA_MEDIO_NORMAL",
    "IN_COMUM_EJA_FUND",
    "IN_COMUM_EJA_MEDIO",
    "IN_COMUM_EJA_PROF",
    "IN_ESP_EXCLUSIVA_EJA_FUND",
    "IN_ESP_EXCLUSIVA_EJA_MEDIO",
    "IN_ESP_EXCLUSIVA_EJA_PROF",
    "IN_EJA_FUNDAMENTAL",
    "IN_EJA_MEDIO",
    "IN_EJA_PROJOVEM",
    "IN_COMUM_PROF",
    "IN_ESP_EXCLUSIVA_PROF",
]

# colunas que poderão ser extraídas da base de matriculas
cols_mat = [
    "NUM_ALUNOS_ATEND_ESCOLA",
    "NUM_ALUNOS_ATEND_OUTRA_ESCOLA",
    "NUM_ALUNOS_ED_COMP_ESCOLA",
    "NUM_ALUNOS_ED_COMP_OUTRA",
    "NUM_ALUNOS_EJA_1A4_APROV_1SEM",
    "NUM_ALUNOS_EJA_1A4_APROV_2SEM",
    "NUM_ALUNOS_EJA_1A4_NOVOS_1SEM",
    "NUM_ALUNOS_EJA_1A4_NOVOS_2SEM",
    "NUM_ALUNOS_EJA_5A8_APROV_1SEM",
    "NUM_ALUNOS_EJA_5A8_APROV_2SEM",
    "NUM_ALUNOS_EJA_5A8_NOVOS_1SEM",
    "NUM_ALUNOS_EJA_5A8_NOVOS_2SEM",
    "NUM_ALUNOS_EJA_MED_APROV_1SEM",
    "NUM_ALUNOS_EJA_MED_APROV_2SEM",
    "NUM_ALUNOS_EJA_MED_NOVOS_1SEM",
    "NUM_ALUNOS_EJA_MED_NOVOS_2SEM",
    "NUM_LIVROS_DEV_2SERIE",
    "NUM_LIVROS_DEV_3SERIE",
    "NUM_LIVROS_DEV_4SERIE",
    "NUM_LIVROS_DEV_5SERIE",
    "NUM_LIVROS_DEV_6SERIE",
    "NUM_LIVROS_DEV_7SERIE",
    "NUM_LIVROS_DEV_8SERIE",
    "NUM_LIVROS_REU_2SERIE",
    "NUM_LIVROS_REU_3SERIE",
    "NUM_LIVROS_REU_4SERIE",
    "NUM_LIVROS_REU_5SERIE",
    "NUM_LIVROS_REU_6SERIE",
    "NUM_LIVROS_REU_7SERIE",
    "NUM_LIVROS_REU_8SERIE",
]

# colunas redundantes com outras da base
cols_red = [
    "CO_ORGAO_REGIONAL",
    "CO_REGIAO",
    "CO_MESORREGIAO",
    "CO_MICRORREGIAO",
    "CO_UF",
    "CO_MUNICIPIO",
    "SIGLA_UF",
]

# dropa as colunas
cols_drop = cols_turma + cols_mat + cols_red
for base in dados.values():
    base.drop(columns=cols_drop, inplace=True, errors="ignore")

In [150]:
# lista todas as colunas disponíveis entre todas as bases
cols = list()
for base in dados.values():
    cols += list(base.columns)
cols = set(cols)

### DT 

In [151]:
# converte as variáveis de data para datetime
for base in dados.values():
    colunas_data = [c for c in base.columns if c.startswith("DT_")]
    for c in colunas_data:
        try:
            base[c] = pd.to_datetime(base[c], format="%d/%m/%Y")
        except ValueError:
            base[c] = pd.to_datetime(base[c], format="%d%b%Y:00:00:00")

### IN 

In [152]:
# preenche bases com colunas IN quando há uma coluna QT
for col in cols:
    if col[:2] == "QT" and f"IN{col[2:]}" in cols:
        for base in dados.values():
            if f"IN{col[2:]}" not in base and col in base:
                base[f"IN{col[2:]}"] = (base[col] > 0).astype("int")
                
# cria a coluna IN_EQUIP_NENHUM
for base in dados.values():
    if "IN_EQUIP_NENHUM" not in base.columns:
        base["IN_EQUIP_NENHUM"] = (
            base.reindex(
                columns=[
                    "IN_EQUIP_PARABOLICA",
                    "IN_EQUIP_SCANNER",
                    "IN_EQUIP_COPIADORA",
                    "IN_EQUIP_IMPRESSORA",
                    "IN_EQUIP_IMPRESSORA_BRAILE",
                    "IN_EQUIP_IMPRESSORA_MULT",
                    "IN_EQUIP_FAX"
                ]
            )
            .sum(axis=1)
            == 0
        ).astype("int")
        
# cria a coluna IN_MATERIAL_PED_NENHUM
for base in dados.values():
    if "IN_MATERIAL_PED_NENHUM" not in base.columns:
        base["IN_MATERIAL_PED_NENHUM"] = (
            base.reindex(columns=[c for c in cols if "IN_MATERIAL_PED_" in c])
            .sum(axis=1)
            == 0
        ).astype("int")
        
# cria a coluna IN_ENERGIA_GERADOR
for base in dados.values():
    if (
        "IN_ENERGIA_GERADOR" not in base 
        and "IN_ENERGIA_RENOVAVEL" in base 
        and "IN_ENERGIA_GERADOR_FOSSIL" in base
    ):
        base["IN_ENERGIA_GERADOR"] = (
            base[["IN_ENERGIA_RENOVAVEL", "IN_ENERGIA_GERADOR_FOSSIL"]].sum(axis=1)
            > 0
        ).astype("int")
        

# cria a coluna IN_DEPENDENCIAS_PNE
for base in dados.values():
    if (
        "IN_DEPENDENCIAS_PNE" not in base 
        and "IN_ACESSIBILIDADE_CORRIMAO" in base 
    ):
        base["IN_DEPENDENCIAS_PNE"] = (
            base[[c for c in base if "IN_ACESSIBILIDADE_" in c]].sum(axis=1)
            > 0
        ).astype("int")
        
# cria a coluna IN_ENERGIA_OUTROS
for base in dados.values():
    if (
        "IN_ENERGIA_OUTROS" not in base 
        and "IN_ENERGIA_INEXISTENTE" in base 
    ):
        base["IN_ENERGIA_OUTROS"] = (
            (
                base[
                    [
                        c for c in base if "IN_ENERGIA" in c 
                        and c != "IN_ENERGIA_INEXISTENTE"
                    ]
                ].sum(axis=1) == 0
            )
            & (base["IN_ENERGIA_INEXISTENTE"] == 0)
        ).astype("int")
        
        
# cria a coluna IN_BANHEIRO
for base in dados.values():
    if (
        "IN_BANHEIRO" not in base 
        and "IN_BANHEIRO_DENTRO_PREDIO" in base 
    ):
        base["IN_BANHEIRO"] = (
            base[[c for c in base if "IN_BANHEIRO_" in c]].sum(axis=1)
            > 0
        ).astype("int")
        

# cria a coluna IN_DESKTOP
for base in dados.values():
    if (
        "IN_DESKTOP" not in base 
        and "QT_DESKTOP_ADM" in base 
    ):
        base["IN_DESKTOP"] = (base["QT_DESKTOP_ADM"] > 0).astype("int")
        
# cria a coluna IN_LOCAL_FUNC_GALPAO
for base in dados.values():
    if (
        "IN_LOCAL_FUNC_GALPAO" not in base 
        and "TP_OCUPACAO_GALPAO" in base 
    ):
        base["IN_LOCAL_FUNC_GALPAO"] = np.where(
            (base["TP_OCUPACAO_GALPAO"] > 0)
            & (base["TP_OCUPACAO_GALPAO"] <= 3),
            1,
            np.where(base["TP_OCUPACAO_GALPAO"] == 0, 0, np.nan)
        )
        
# cria a coluna IN_LINGUA_INDIGENA e IN_LINGUA_PORTUGUESA
for base in dados.values():
    if (
        "IN_LINGUA_INDIGENA" not in base 
        and "TP_INDIGENA_LINGUA" in base 
    ):
        base["IN_LINGUA_INDIGENA"] = (
            base["TP_INDIGENA_LINGUA"].isin([1, 3])
        ).astype("int")
    
    if (
        "IN_LINGUA_PORTUGUESA" not in base 
        and "TP_INDIGENA_LINGUA" in base 
    ):
        base["IN_LINGUA_PORTUGUESA"] = (
            base["TP_INDIGENA_LINGUA"].isin([2, 3])
        ).astype("int")
        
# cria a coluna IN_BIBLIOTECA_SALA_LEITURA
for base in dados.values():
    if (
        "IN_BIBLIOTECA_SALA_LEITURA" not in base 
        and "IN_BIBLIOTECA" in base 
        and "IN_SALA_LEITURA" in base
    ):
        base["IN_BIBLIOTECA_SALA_LEITURA"] = (
            (base["IN_BIBLIOTECA"] == 1)
            | (base["IN_SALA_LEITURA"] == 1)
        ).astype("int")
        

# cria a coluna IN_QUADRA_ESPORTES
for base in dados.values():
    if (
        "IN_QUADRA_ESPORTES" not in base 
        and "IN_QUADRA_ESPORTES_COBERTA" in base 
        and "IN_QUADRA_ESPORTES_DESCOBERTA" in base
    ):
        base["IN_QUADRA_ESPORTES"] = (
            (base["IN_QUADRA_ESPORTES_COBERTA"] == 1)
            | (base["IN_QUADRA_ESPORTES_DESCOBERTA"] == 1)
        ).astype("int")

# cria a coluna IN_LOCAL_FUNC_PREDIO_ESCOLAR
for base in dados.values():
    if (
        "IN_LOCAL_FUNC_PREDIO_ESCOLAR" not in base 
        and "TP_OCUPACAO_PREDIO_ESCOLAR" in base 
    ):
        base["IN_LOCAL_FUNC_PREDIO_ESCOLAR"] = (
            (base["TP_OCUPACAO_PREDIO_ESCOLAR"] > 0)
        ).astype("int")
        
# corrige a coluna IN_BIBLIOTECA
for base in dados.values():
    if (
        "IN_SALA_LEITURA" not in base 
        and "IN_BIBLIOTECA" in base 
    ):
        base.rename(columns={"IN_BIBLIOTECA": "IN_BIBLIOTECA_SALA_LEITURA"}, inplace=True)
        
# cria a coluna IN_AGUA_POTAVEL
for base in dados.values():
    if (
        "IN_AGUA_POTAVEL" not in base 
        and "IN_AGUA_FILTRADA" in base 
    ):
        base["IN_AGUA_POTAVEL"] = (
            (base["IN_AGUA_FILTRADA"] == 2)
        ).astype("int")
        
# substítui o valor 9 pelo valor nulo
for base in dados.values():
    for c in base:
        if c.startswith("IN_"):
            base[c] = base[c].replace({9: np.nan})

###  TP

In [153]:
# cria a coluna TP_INDIGENA_LINGUA
for base in dados.values():
    if (
        "TP_INDIGENA_LINGUA" not in base 
        and "IN_LINGUA_INDIGENA" in base 
        and "IN_LINGUA_PORTUGUESA" in base 
    ):
        base["TP_INDIGENA_LINGUA"] = np.where(
            (base["IN_LINGUA_INDIGENA"] == 1)
            & (base["IN_LINGUA_PORTUGUESA"] == 0),
            1,
            np.where(
                (base["IN_LINGUA_INDIGENA"] == 0)
                & (base["IN_LINGUA_PORTUGUESA"] == 1),
                2, 
                np.where(
                    (base["IN_LINGUA_INDIGENA"] == 1)
                    & (base["IN_LINGUA_PORTUGUESA"] == 1),
                    3,
                    np.where(
                        (
                            base["TP_SITUACAO_FUNCIONAMENTO"].astype("str")
                            .isin(["1", "1.0", "EM ATIVIDADE"])
                        ) & (base["IN_EDUCACAO_INDIGENA"] == 0),
                        0,
                        np.nan
                    )
                )
            )
        )

# Corrige o campo de TP_OCUPACAO_GALPAO
for base in dados.values():
    if "TP_OCUPACAO_GALPAO" in base:
        if base["TP_OCUPACAO_GALPAO"].max() == 1:
            base.drop(columns={"TP_OCUPACAO_GALPAO"}, inplace=True)

# Corrige o campo de TP_OCUPACAO_GALPAO
for base in dados.values():
    if "TP_OCUPACAO_PREDIO_ESCOLAR" in base:
        if base["TP_OCUPACAO_PREDIO_ESCOLAR"].max() == 1:
            base.drop(columns={"TP_OCUPACAO_PREDIO_ESCOLAR"}, inplace=True)
        
        
# cria o depara de campos para categorias
depara = {
    "TP_CATEGORIA_ESCOLA_PRIVADA": {
        1: "PARTICULAR", 
        2: "COMUNITÁRIA", 
        3: "CONFESSIONAL", 
        4: "FILANTRÓPICA"
    },
    "TP_REDE_LOCAL": {
        0: "NÃO HÁ REDE LOCAL INTERLIGANDO COMPUTADORES",
        1: "A CABO",
        2: "WIRELESS",
        3: "A CABO E WIRELESS",
        9: "NÃO INFORMADO",
    },
    "TP_REGULAMENTACAO": {
        0: "NÃO",
        1: "SIM",
        2: "EM TRAMITAÇÃO",
    },
    "TP_RESPONSAVEL_REGULAMENTACAO": {
        1: "FEDERAL",
        2: "ESTADUAL",
        3: "MUNICIPAL",
        4: "ESTADUAL E MUNICIPAL",
        5: "FEDERAL E ESTADUAL",
        6: "FEDERAL, ESTADUAL E MUNICIPAL",
        9: "NÃO INFORMADO",
    },
    "TP_OCUPACAO_GALPAO": {
        0: "NÃO",
        1: "PRÓPRIO",
        2: "ALUGADO",
        3: "CEDIDO",
        4: "NÃO INFORMADO",
        9: "NÃO INFORMADO"
    },
    "TP_LOCALIZACAO": {
        1: "URBANA",
        2: "RURAL",
    },
    "TP_CONVENIO_PODER_PUBLICO": {
        1: "MUNICIPAL",
        2: "ESTADUAL",
        3: "ESTADUAL E MUNICIPAL",
    },
    "TP_PROPOSTA_PEDAGOGICA": {
        0: "NÃO",
        1: "SIM",
        2: "A ESCOLA NÃO POSSUI PROJETO POLÍTICO PEDAGÓGICO/PROPOSTA PEDAGÓGICA",
        9: "NÃO INFORMADO",
    },
    "TP_DEPENDENCIA": {
        1: "FEDERAL",
        2: "ESTADUAL",
        3: "MUNICIPAL",
        4: "PRIVADA",
    },
    "TP_LOCALIZACAO_DIFERENCIADA": {
        0: "A ESCOLA NÃO ESTÁ EM ÁREA DE LOCALIZAÇÃO DIFERENCIADA",
        1: "ÁREA DE ASSENTAMENTO",
        2: "TERRA INDÍGENA",
        3: "ÁREA REMANESCENTE DE QUILOMBOS",
        4: "ÁREA DE ASSENTAMENTO",
        5: "TERRA INDÍGENA",
        6: "ÁREA REMANESCENTE DE QUILOMBOS",
    },
    "TP_INDIGENA_LINGUA": {
        0: "SEM EDUCAÇÃO INDÍGENA",
        1: "SOMENTE EM LÍNGUA INDÍGENA",
        2: "SOMENTE EM LÍNGUA PORTUGUESA",
        3: "EM LÍNGUA INDÍGENA E EM LÍNGUA PORTUGUESA",
    },
    "TP_SITUACAO_FUNCIONAMENTO": {
        1: "EM ATIVIDADE", 
        2: "PARALISADA", 
        3: "EXTINTA", 
        4: "EXTINTA",
    },
    "TP_OCUPACAO_PREDIO_ESCOLAR": {
        0: "NÃO",
        1: "PRÓPRIO",
        2: "ALUGADO",
        3: "CEDIDO",
    }
}

preencher_tp = {
    "TP_CATEGORIA_ESCOLA_PRIVADA": "PÚBLICA",
    "TP_REDE_LOCAL": "NÃO INFORMADO",
    "TP_REGULAMENTACAO": None,
    "TP_RESPONSAVEL_REGULAMENTACAO": None,
    "TP_OCUPACAO_GALPAO": "NÃO",
    "TP_LOCALIZACAO": None,
    "TP_CONVENIO_PODER_PUBLICO": "PÚBLICA",
    "TP_PROPOSTA_PEDAGOGICA": None,
    "TP_DEPENDENCIA": None,
    "TP_LOCALIZACAO_DIFERENCIADA": None,
    "TP_INDIGENA_LINGUA": None,
    "TP_SITUACAO_FUNCIONAMENTO": None,
    "TP_OCUPACAO_PREDIO_ESCOLAR": "NÃO",
}

# converte a coluna para tipo categórico
for k, base in dados.items():
    for c, d in depara.items():
        if c in base:
            # lista os valores a serem categorizados
            vals = list(d.values())
            
            # obtém os valores 
            unicos = set(base[c].dropna().replace(d))
            esperado = set(vals)
            
            # verifica que não há nenhum erro com os dados a serem preenchidos
            if not esperado.issuperset(unicos):
                raise ValueError(
                    f"A coluna {c} da base {k} possuí os valores "
                    f"{unicos - esperado} a mais"
                )
            
            # cria o tipo categórico
            if preencher_tp[c] is not None:
                vals += [preencher_tp[c]]
            cat = pd.Categorical(vals).dtype
            
            # realiza a conversão da coluna
            base[c] = base[c].replace(d).astype(cat)
            

# vamos converter as variáveis indicadores de escolas particulares
# em variáveis TP, uma vez que teremos uma terceira opção informando
# que a escola é pública
particular = [
    "TP_CATEGORIA_ESCOLA_PRIVADA",
    "IN_CONVENIADA_PP",
    "IN_MANT_ESCOLA_PRIVADA_EMP",
    "IN_MANT_ESCOLA_PRIVADA_ONG",
    "IN_MANT_ESCOLA_PRIVADA_OSCIP",
    "IN_MANT_ESCOLA_PRIV_ONG_OSCIP",
    "IN_MANT_ESCOLA_PRIVADA_SIND",
    "IN_MANT_ESCOLA_PRIVADA_SIST_S",
    "IN_MANT_ESCOLA_PRIVADA_S_FINS",
    "TP_CATEGORIA_ESCOLA_PRIVADA",
]

for c in particular:
    for base in dados.values():
        if c in base:
            base.rename(columns={c: f"TP{c[2:]}"}, inplace=True)
            base[f"TP{c[2:]}"] = base[f"TP{c[2:]}"].replace({0: "NÃO", 1: "SIM"})
            base[f"TP{c[2:]}"] = np.where(
                (base["TP_SITUACAO_FUNCIONAMENTO"] == "EM ATIVIDADE")
                & (base["TP_DEPENDENCIA"] == "PRIVADA"),
                base[f"TP{c[2:]}"],
                np.where(
                    (base["TP_SITUACAO_FUNCIONAMENTO"] == "EM ATIVIDADE")
                    & ~(base["TP_DEPENDENCIA"] == "PRIVADA"),
                    base[f"TP{c[2:]}"].fillna("PÚBLICA"),
                    np.nan
                )
            )
            base[f"TP{c[2:]}"] = base[f"TP{c[2:]}"].astype("category")

### QT 

In [154]:
# substítui o valor 88888 pelo valor nulo
col_88888 = [
    "QT_EQUIP_DVD",
    "QT_EQUIP_SOM",
    "QT_EQUIP_TV",
    "QT_EQUIP_MULTIMIDIA"
]
for k, base in dados.items():
    if k >= "2019.zip":
        for c in col_88888:
            if c in base:
                base[c] = base[c].replace({88888: np.nan})

### Concatenação 

In [297]:
atemporal = [
    "CO_IES_OFERTANTE",
    "CO_DISTRITO",
    "NO_ENTIDADE",
    "TP_LOCALIZACAO",
    "TP_LOCALIZACAO_DIFERENCIADA",
]

dados_comp = pd.concat(
    [
        base.loc[lambda f: f["TP_SITUACAO_FUNCIONAMENTO"] == "EM ATIVIDADE"]
        .drop(columns=atemporal + ["TP_SITUACAO_FUNCIONAMENTO"], errors="ignore")
        for base in dados.values()
    ]
)

dados_atemp = pd.concat(
    [base.reindex(columns=["CO_ENTIDADE"] + atemporal) for base in dados.values()]
).drop_duplicates(subset=["CO_ENTIDADE"], keep="last")

### Preenchimento de Nulos 

In [299]:
fbfill = [
    "TP_DEPENDENCIA",
    "TP_CATEGORIA_ESCOLA_PRIVADA",
    "TP_CONVENIADA_PP",
    "TP_MANT_ESCOLA_PRIVADA_EMP",
    "TP_MANT_ESCOLA_PRIVADA_ONG",
    "TP_MANT_ESCOLA_PRIVADA_SIND", # não existem o censo de 2007 e 2008
    "TP_MANT_ESCOLA_PRIVADA_S_FINS", # não existem o censo de 2007 e 2008
    "TP_REGULAMENTACAO",
    "IN_LOCAL_FUNC_PREDIO_ESCOLAR",
    "IN_LOCAL_FUNC_PRISIONAL_SOCIO",
    "IN_LOCAL_FUNC_GALPAO",
    "IN_LOCAL_FUNC_SALAS_OUTRA_ESC",
    "IN_LOCAL_FUNC_OUTROS",
    "IN_PREDIO_COMPARTILHADO",
    "IN_AGUA_REDE_PUBLICA",
    "IN_AGUA_POCO_ARTESIANO",
    "IN_AGUA_CACIMBA",
    "IN_AGUA_FONTE_RIO",
    "IN_AGUA_INEXISTENTE",
    "IN_ENERGIA_REDE_PUBLICA",
    "IN_ENERGIA_INEXISTENTE",
    "IN_ENERGIA_GERADOR",
    "IN_ENERGIA_OUTROS",
    "IN_ESGOTO_REDE_PUBLICA",
    "IN_ESGOTO_FOSSA",
    "IN_ESGOTO_INEXISTENTE",
    "IN_LIXO_SERVICO_COLETA",
    "IN_LIXO_QUEIMA",
    "IN_LIXO_DESCARTA_OUTRA_AREA",
    "IN_TRATAMENTO_LIXO_RECICLAGEM",
    "IN_LIXO_ENTERRA",
    "IN_LIXO_DESTINO_FINAL_PUBLICO",
    "IN_BANHEIRO_PNE",
    "IN_BIBLIOTECA_SALA_LEITURA",
    "IN_COZINHA",
    "IN_LABORATORIO_CIENCIAS",
    "IN_LABORATORIO_INFORMATICA",
    "IN_PARQUE_INFANTIL",
    "IN_QUADRA_ESPORTES",
    "IN_SALA_DIRETORIA",
    "IN_SALA_PROFESSOR",
    "IN_SALA_ATENDIMENTO_ESPECIAL",
    "IN_DEPENDENCIAS_OUTRAS",
    "QT_SALAS_UTILIZADAS",
    "IN_EQUIP_TV",
    "IN_EQUIP_DVD",
    "IN_EQUIP_PARABOLICA",
    "IN_EQUIP_COPIADORA",
    "IN_EQUIP_IMPRESSORA",
    "IN_DESKTOP",
    "IN_INTERNET",
    "IN_ALIMENTACAO",
    "IN_EDUCACAO_INDIGENA",
    "TP_INDIGENA_LINGUA",
    "IN_MATERIAL_PED_ETNICO",
    "IN_MATERIAL_PED_NENHUM",
    "IN_MATERIAL_PED_INDIGENA",
    "IN_LOCAL_FUNC_SALAS_EMPRESA", # não existem o censo de 2019 e 2020
    "IN_LOCAL_FUNC_TEMPLO_IGREJA", # não existem o censo de 2019 e 2020
    "IN_LOCAL_FUNC_CASA_PROFESSOR", # não existem o censo de 2019 e 2020
    "IN_BANHEIRO_FORA_PREDIO", # não existem o censo de 2019 e 2020
    "IN_BANHEIRO_DENTRO_PREDIO", # não existem o censo de 2019 e 2020
    "IN_EQUIP_RETRO", # não existem o censo de 2019 e 2020
    "IN_EQUIP_VIDEOCASSETE", # não existem o censo de 2019 e 2020
    "IN_BANHEIRO_EI", # não existem o censo de 2007 e 2008
    "IN_SALA_LEITURA", # não existem o censo de 2007 e 2008
    "IN_BIBLIOTECA", # não existem o censo de 2007 e 2008
    "IN_BANDA_LARGA", # não existem em 2007
    "IN_BERCARIO", # não existem em 2007, 2008, 2019, 2020
    "IN_QUADRA_ESPORTES_COBERTA", # não existem antes de 2011
    "IN_QUADRA_ESPORTES_DESCOBERTA", # não existem antes de 2011
    "TP_OCUPACAO_PREDIO_ESCOLAR", # não existem antes de 2011
    "IN_PATIO_DESCOBERTO", # não existem antes de 2012
    "TP_MANT_ESCOLA_PRIVADA_SIST_S", # não existem antes de 2012
    "IN_AREA_VERDE", # não existem antes de 2012
    "IN_DORMITORIO_PROFESSOR", # não existem antes de 2012
    "IN_PATIO_COBERTO", # não existem antes de 2012
    "IN_AUDITORIO", # não existem antes de 2012
    "IN_ALMOXARIFADO", # não existem antes de 2012
    "IN_DESPENSA", # não existem antes de 2012
    "IN_REFEITORIO", # não existem antes de 2012
    "IN_BANHEIRO_CHUVEIRO", # não existem antes de 2012
    "IN_SECRETARIA", # não existem antes de 2012
    "IN_DORMITORIO_ALUNO", # não existem antes de 2012
    "IN_EQUIP_SOM", # não existem antes de 2012
    "IN_EQUIP_MULTIMIDIA", # não existem antes de 2012
    "IN_LOCAL_FUNC_SOCIOEDUCATIVO", # não existem antes de 2012
    "IN_EQUIP_FAX", # só existe de 2012 a 2018
    "IN_FINAL_SEMANA", # só existe de 2012 a 2018
    "IN_BRASIL_ALFABETIZADO", # só existe de 2012 a 2018
    "IN_EQUIP_FOTO", # só existe de 2012 a 2018
    "IN_LAVANDERIA", # só existe de 2012 a 2018
    "IN_EQUIP_IMPRESSORA_MULT", # só existe de 2015 em diante
    "TP_OCUPACAO_GALPAO", # só existe de 2012 em diante
]

# preenchimento com valores históricos
dados_comp.sort_values(by=["CO_ENTIDADE", "NU_ANO_CENSO"], inplace=True)
dados_comp.reset_index(drop=True, inplace=True)
for c in fbfill:
    dados_comp[c] = (
        dados_comp.groupby(["CO_ENTIDADE"])[c]
        .ffill()
        .values
    )
    dados_comp[c] = (
        dados_comp.groupby(["CO_ENTIDADE"])[c]
        .bfill()
        .values
    )
    
# remove colunas que são redundantes
remover = ["IN_AGUA_FILTRADA", "IN_LINGUA_INDIGENA", "IN_LINGUA_PORTUGUESA"]
dados_comp.drop(columns=remover, inplace=True)

# corrige variáveis de escolas privadas
var_particular = [
    "TP_MANT_ESCOLA_PRIVADA_SIND",
    "TP_CATEGORIA_ESCOLA_PRIVADA",
    "TP_CONVENIADA_PP",
    "TP_MANT_ESCOLA_PRIVADA_EMP",
    "TP_MANT_ESCOLA_PRIVADA_ONG",
    "TP_MANT_ESCOLA_PRIVADA_SIND",
    "TP_MANT_ESCOLA_PRIVADA_S_FINS",
]
for c in var_particular:
    dados_comp[c] = np.where(
        (dados_comp[c] == "PÚBLICA")
        & (dados_comp["TP_DEPENDENCIA"] == "PRIVADA"),
        np.nan,
        dados_comp[c]
    )
    
# corrige variáveis relacionadas a galpões
var_galpao = ["TP_OCUPACAO_GALPAO"]
for c in var_galpao:
    dados_comp[c] = np.where(
        ~(dados_comp[c] == "NÃO")
        & ~(dados_comp["IN_LOCAL_FUNC_GALPAO"] == 0),
        np.nan,
        dados_comp[c]
    )
    
# preenche nulos
preencher = {
    "TP_INDIGENA_LINGUA": "SEM EDUCAÇÃO INDÍGENA"
}
for c, p in preencher.items():
    dados_comp[c] = dados_comp[c].fillna(p)

---